In [11]:
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from osgeo import gdal
import geopandas as gpd
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
from pyproj import CRS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pyproj
import netCDF4 as nc
import glob
import pickle 
import random
from tqdm import tqdm  
import datetime as datetime
import gc
import os
import h5py
from scipy.stats import pearson3, norm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [35]:
wind = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Wind/Ian_Florida.csv")
wind.head()

,ID,Center_Latitude,Center_Longitude,Vmax
0,0,32.224223,-87.451826,14.019410
1,1,32.219746,-87.452504,14.018406
2,2,32.215270,-87.453181,14.027005
3,3,32.210793,-87.453858,14.026000
4,4,32.206316,-87.454535,14.039413


In [36]:
wind.shape

(1048575, 4)

In [6]:
#wind = wind[wind["Vmax"]>20]

In [37]:
night_month = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Nightlights_Monthly/Florida_Ian/Zonal_Statistics_Monthly_Ian.csv")
night_month.head()

,id,left,top,right,bottom,_count,_sum,_mean
0,1,1.184478e+06,-1.329093e+06,1.184978e+06,-1.329593e+06,NaN,NaN,NaN
1,2,1.184478e+06,-1.329593e+06,1.184978e+06,-1.330093e+06,NaN,NaN,NaN
2,3,1.184478e+06,-1.330093e+06,1.184978e+06,-1.330593e+06,NaN,NaN,NaN
3,4,1.184478e+06,-1.330593e+06,1.184978e+06,-1.331093e+06,NaN,NaN,NaN
4,5,1.184478e+06,-1.331093e+06,1.184978e+06,-1.331593e+06,NaN,NaN,NaN


In [38]:
#night_month = night_month.merge(wind,left_on="id",right_on="ID")
#night_month.head()

In [39]:
files_list = sorted(glob.glob("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Rainfall/Ian/September_28.nc4"))
files_list[0]

'C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Rainfall/Ian/September_28.nc4'

In [40]:
# Open the .nc4 file
file_path = files_list[0]
dataset = nc.Dataset(file_path)

dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    CDI: Climate Data Interface version 1.9.8 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    BeginDate: 2020-09-28
    BeginTime: 00:00:00.000Z
    EndDate: 2020-09-28
    EndTime: 23:59:59.999Z
    FileHeader: StartGranuleDateTime=2020-09-28T00:00:00.000Z;
StopGranuleDateTime=2020-09-28T23:59:59.999Z
    InputPointer: 3B-HHR.MS.MRG.3IMERG.20200928-S000000-E002959.0000.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S003000-E005959.0030.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S010000-E012959.0060.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S013000-E015959.0090.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S020000-E022959.0120.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S023000-E025959.0150.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S030000-E032959.0180.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S033000-E035959.0210.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S040000-E042959.0240.V06B.HDF5;3B-HHR.MS.MRG.3IMERG.20200928-S0430

In [41]:
# Define the coordinates in New Jersey as a DataFrame
fl_coordinates_df = pd.read_csv("C:/users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Temp/latlong_Florida_degrees.csv")
fl_coordinates_df.head()

,Unnamed: 0,Center_Latitude,Center_Longitude
0,0,32.224223,-87.451826
1,1,32.219746,-87.452504
2,2,32.215270,-87.453181
3,3,32.210793,-87.453858
4,4,32.206316,-87.454535


In [42]:
import netCDF4 as nc
import numpy as np
from scipy.spatial import cKDTree
import pandas as pd

# Open the netCDF file
dataset = nc.Dataset(files_list[0])

# Read the variable
precipitation = dataset.variables['precipitationCal'][:]

# Get the longitude and latitude values
lon = dataset.variables['lon'][:]
lat = dataset.variables['lat'][:]

# Reshape the longitude and latitude arrays to have the same dimensions
lon_2d, lat_2d = np.meshgrid(lon, lat)

# Create a KDTree from the reshaped longitude and latitude arrays
tree = cKDTree(np.column_stack((lon_2d.ravel(), lat_2d.ravel())))

# Define the coordinates in New Jersey as a DataFrame
#nj_coordinates_df = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Temp/latlong_NJ_degrees.csv")

# Find the nearest grid points to the New Jersey coordinates
distances, indices = tree.query(np.column_stack((fl_coordinates_df['Center_Longitude'], fl_coordinates_df['Center_Latitude'])))

# Filter out indices that are out of bounds
valid_indices = np.logical_and(indices >= 0, indices < lon.shape[0] * lat.shape[0])

# Get the valid indices and rainfall values
valid_indices = valid_indices.nonzero()[0]
rainfall_values = precipitation.ravel()[indices[valid_indices]]

# Create a new DataFrame with the valid indices and rainfall values
fl_coordinates_df_filtered = fl_coordinates_df.loc[valid_indices].copy()
fl_coordinates_df_filtered['Rainfall'] = rainfall_values

fl_coordinates_df_filtered.head()

,Unnamed: 0,Center_Latitude,Center_Longitude,Rainfall
0,0,32.224223,-87.451826,62.577663
1,1,32.219746,-87.452504,62.577663
2,2,32.215270,-87.453181,62.577663
3,3,32.210793,-87.453858,62.577663
4,4,32.206316,-87.454535,62.577663


In [43]:
rainfall = fl_coordinates_df_filtered.copy()
rainfall = rainfall.rename(columns={'Unnamed: 0': 'id'})
rainfall.head()

,id,Center_Latitude,Center_Longitude,Rainfall
0,0,32.224223,-87.451826,62.577663
1,1,32.219746,-87.452504,62.577663
2,2,32.215270,-87.453181,62.577663
3,3,32.210793,-87.453858,62.577663
4,4,32.206316,-87.454535,62.577663


In [44]:
landcover = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Zonal_Statistics_Florida_Landcover.csv")
landcover.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_3,HISTO_4,HISTO_6,HISTO_7,HISTO_8,HISTO_9,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA
0,1,1.184478e+06,-1.329093e+06,1.184978e+06,-1.329593e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1.184478e+06,-1.329593e+06,1.184978e+06,-1.330093e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.184478e+06,-1.330093e+06,1.184978e+06,-1.330593e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1.184478e+06,-1.330593e+06,1.184978e+06,-1.331093e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,1.184478e+06,-1.331093e+06,1.184978e+06,-1.331593e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
landcover.columns

Index(['id', 'left', 'top', 'right', 'bottom', 'HISTO_0', 'HISTO_1', 'HISTO_3',
       'HISTO_4', 'HISTO_6', 'HISTO_7', 'HISTO_8', 'HISTO_9', 'HISTO_10',
       'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'HISTO_NODATA'],
      dtype='object')

In [46]:
histo_columns = ['HISTO_0', 'HISTO_1', 'HISTO_3',
       'HISTO_4', 'HISTO_6', 'HISTO_7', 'HISTO_8', 'HISTO_9', 'HISTO_10',
       'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'HISTO_NODATA']

# Convert HISTO columns to percentages
landcover[histo_columns] = landcover[histo_columns].div(landcover[histo_columns].sum(axis=1), axis=0) * 100
landcover.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_3,HISTO_4,HISTO_6,HISTO_7,HISTO_8,HISTO_9,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA
0,1,1.184478e+06,-1.329093e+06,1.184978e+06,-1.329593e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.184478e+06,-1.329593e+06,1.184978e+06,-1.330093e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1.184478e+06,-1.330093e+06,1.184978e+06,-1.330593e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1.184478e+06,-1.330593e+06,1.184978e+06,-1.331093e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1.184478e+06,-1.331093e+06,1.184978e+06,-1.331593e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
landcover.columns

Index(['id', 'left', 'top', 'right', 'bottom', 'HISTO_0', 'HISTO_1', 'HISTO_3',
       'HISTO_4', 'HISTO_6', 'HISTO_7', 'HISTO_8', 'HISTO_9', 'HISTO_10',
       'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'HISTO_NODATA'],
      dtype='object')

In [58]:
landcover = landcover[['id', 'HISTO_0', 'HISTO_1', 'HISTO_3',
       'HISTO_4', 'HISTO_6', 'HISTO_7', 'HISTO_8', 'HISTO_9', 'HISTO_10',
       'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18']]

landcover.head()

,id,HISTO_0,HISTO_1,HISTO_3,HISTO_4,HISTO_6,HISTO_7,HISTO_8,HISTO_9,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
night_28 = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Nightlights_Monthly/Florida_Ian/Zonal_28th.csv")
night_28.head()

,id,left,top,right,bottom,_count,_sum,_mean
0,1,1.184478e+06,-1.329093e+06,1.184978e+06,-1.329593e+06,NaN,NaN,NaN
1,2,1.184478e+06,-1.329593e+06,1.184978e+06,-1.330093e+06,NaN,NaN,NaN
2,3,1.184478e+06,-1.330093e+06,1.184978e+06,-1.330593e+06,NaN,NaN,NaN
3,4,1.184478e+06,-1.330593e+06,1.184978e+06,-1.331093e+06,NaN,NaN,NaN
4,5,1.184478e+06,-1.331093e+06,1.184978e+06,-1.331593e+06,NaN,NaN,NaN


In [48]:
night_28 = night_28[["id","_mean"]]

In [49]:
night_month.head()

,id,left,top,right,bottom,_count,_sum,_mean
0,1,1.184478e+06,-1.329093e+06,1.184978e+06,-1.329593e+06,NaN,NaN,NaN
1,2,1.184478e+06,-1.329593e+06,1.184978e+06,-1.330093e+06,NaN,NaN,NaN
2,3,1.184478e+06,-1.330093e+06,1.184978e+06,-1.330593e+06,NaN,NaN,NaN
3,4,1.184478e+06,-1.330593e+06,1.184978e+06,-1.331093e+06,NaN,NaN,NaN
4,5,1.184478e+06,-1.331093e+06,1.184978e+06,-1.331593e+06,NaN,NaN,NaN


In [50]:
night_month = night_month[["id","_mean"]]
night_month = night_month.rename(columns={'_mean': 'nightlight_prev'})

In [51]:
outage = night_28.merge(night_month,on="id")
outage.head()

,id,_mean,nightlight_prev
0,1,NaN,NaN
1,2,NaN,NaN
2,3,NaN,NaN
3,4,NaN,NaN
4,5,NaN,NaN


In [52]:
outage["perc_outage"] = (outage["nightlight_prev"] - outage["_mean"])/outage["nightlight_prev"] * 100
outage.head()

,id,_mean,nightlight_prev,perc_outage
0,1,NaN,NaN,NaN
1,2,NaN,NaN,NaN
2,3,NaN,NaN,NaN
3,4,NaN,NaN,NaN
4,5,NaN,NaN,NaN


In [53]:
outage = outage.dropna(subset=['perc_outage'])
outage.head()

,id,_mean,nightlight_prev,perc_outage
5361,5362,0.699138,0.498923,-4.012951e+01
5362,5363,0.307031,0.002193,-1.390318e+04
5363,5364,0.500000,0.000000,-inf
7046,7047,0.300393,0.000000,-inf
7047,7048,0.686503,0.023106,-2.871091e+03


In [54]:
outage.shape

(91076, 4)

In [55]:
outage = outage[outage["perc_outage"]>0]
outage.head()

,id,_mean,nightlight_prev,perc_outage
10394,10395,0.400000,0.600000,33.333333
10395,10396,0.400000,0.600000,33.333333
12081,12082,0.400000,0.600000,33.333333
12082,12083,0.463428,0.578857,19.940920
12101,12102,0.310934,0.437938,29.000457


In [56]:
outage.shape

(5437, 4)

In [59]:
wind.head()

,ID,Center_Latitude,Center_Longitude,Vmax
0,0,32.224223,-87.451826,14.019410
1,1,32.219746,-87.452504,14.018406
2,2,32.215270,-87.453181,14.027005
3,3,32.210793,-87.453858,14.026000
4,4,32.206316,-87.454535,14.039413


In [60]:
reg_ian = outage.merge(wind,left_on="id",right_on="ID")
reg_ian = reg_ian.merge(landcover,on="id")
reg_ian = reg_ian.merge(rainfall,on="id")
reg_ian.head()

,id,_mean,nightlight_prev,perc_outage,ID,Center_Latitude_x,Center_Longitude_x,Vmax,HISTO_0,HISTO_1,HISTO_3,HISTO_4,HISTO_6,HISTO_7,HISTO_8,HISTO_9,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,Center_Latitude_y,Center_Longitude_y,Rainfall
0,10395,0.400000,0.600000,33.333333,10395,30.997007,-87.601278,15.065447,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,2.205882,0.0,0.0,0.0,0.0,30.997007,-87.601278,94.09639
1,10396,0.400000,0.600000,33.333333,10396,30.992522,-87.601925,15.080445,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,30.992522,-87.601925,94.09639
2,12082,0.400000,0.600000,33.333333,12082,30.996357,-87.596108,15.082678,0.0,0.0,10.726644,0.000000,0.0,0.346021,0.0,0.692042,0.0,61.245675,0.0,0.0,0.0,0.0,30.996357,-87.596108,94.09639
3,12083,0.463428,0.578857,19.940920,12083,30.991872,-87.596755,15.081487,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,69.117647,0.0,0.0,0.0,0.0,30.991872,-87.596755,94.09639
4,12102,0.310934,0.437938,29.000457,12102,30.906647,-87.609024,15.145514,0.0,0.0,11.764706,3.114187,0.0,29.065744,0.0,0.000000,0.0,56.055363,0.0,0.0,0.0,0.0,30.906647,-87.609024,94.09639


In [61]:
reg_ian.columns

Index(['id', '_mean', 'nightlight_prev', 'perc_outage', 'ID',
       'Center_Latitude_x', 'Center_Longitude_x', 'Vmax', 'HISTO_0', 'HISTO_1',
       'HISTO_3', 'HISTO_4', 'HISTO_6', 'HISTO_7', 'HISTO_8', 'HISTO_9',
       'HISTO_10', 'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'Center_Latitude_y', 'Center_Longitude_y', 'Rainfall'],
      dtype='object')

In [65]:
reg_ian = reg_ian[['nightlight_prev', 'perc_outage', 'Vmax', 'HISTO_0', 'HISTO_1',
       'HISTO_3', 'HISTO_4', 'HISTO_6', 'HISTO_7', 'HISTO_8', 'HISTO_9',
       'HISTO_10', 'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'Rainfall']]

In [68]:
nan_count = reg_ian.isna().sum()
print(nan_count)

nightlight_prev    0
perc_outage        0
Vmax               0
HISTO_0            0
HISTO_1            0
HISTO_3            0
HISTO_4            0
HISTO_6            0
HISTO_7            0
HISTO_8            0
HISTO_9            0
HISTO_10           0
HISTO_14           0
HISTO_15           0
HISTO_16           0
HISTO_17           0
HISTO_18           0
Rainfall           0
dtype: int64


In [69]:
df = reg_ian.copy()

#df = df.drop("HISTO_NODATA", axis=1)
df = df[df["Vmax"]>20]

# Separate independent and dependent variables
X = df.drop('perc_outage', axis=1)  # Independent variables
y = df['perc_outage']  # Dependent variable

# Standardize independent variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create and fit the regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Print the model coefficients
print("Model Coefficients:")
for feature, coef in zip(X.columns, reg_model.coef_):
    print(feature, ":", coef)

# Predict on the test set
y_pred = reg_model.predict(X_test)

# Calculate R2 score on the test set
r2 = r2_score(y_test, y_pred)

# Print the R2 score
print("R2 score on the test set:", r2)

Model Coefficients:
nightlight_prev : -5.248350556562735
Vmax : -17.55352861136058
HISTO_0 : 2.1760371282653068e-14
HISTO_1 : 5.684341886080802e-14
HISTO_3 : -2.3575644483433984
HISTO_4 : 3.375077994860476e-14
HISTO_6 : -1.4210854715202004e-14
HISTO_7 : 17.015966089895752
HISTO_8 : 1.7763568394002505e-15
HISTO_9 : 25.30785134528759
HISTO_10 : 0.0
HISTO_14 : 3.2281980190020407
HISTO_15 : 0.0
HISTO_16 : -31.64140904672439
HISTO_17 : 3.277362953935104
HISTO_18 : -21.755070640536726
Rainfall : -9.254708758601986
R2 score on the test set: 0.29723182783412905


In [70]:
import statsmodels.api as sm
import pandas as pd

# Assuming your dataframe is named 'df' and the target variable is '_mean'
target = 'perc_outage'
independent_vars = [col for col in df.columns if col != target]

# Fit the Poisson GLM
X = df[independent_vars]
X = sm.add_constant(X)  # Add a constant term for the intercept
y = df[target]

poisson_model = sm.GLM(y, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Print the summary of the model
print(poisson_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            perc_outage   No. Observations:                   15
Model:                            GLM   Df Residuals:                        4
Model Family:                 Poisson   Df Model:                           10
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -45.916
Date:                Sat, 24 Jun 2023   Deviance:                       19.359
Time:                        17:33:40   Pearson chi2:                     18.2
No. Iterations:                     5   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              37.7520     14.928     